In [1]:
! git clone https://github.com/facebookresearch/fastText.git
! pip3 install fastText/.

fatal: destination path 'fastText' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./fastText
  Preparing metadata (setup.py) ... done
  Created wheel for fasttext: filename=fasttext-0.9.2-cp38-cp38-linux_x86_64.whl size=4391196 sha256=45dff53282a117c05ad6697d5bd64727010d2c0edc689a2bd07ad5a100ce766c
  Stored in directory: /tmp/pip-ephem-wheel-cache-n9n5pn12/wheels/a4/2f/6a/91d479a807787e092f667baec5df08801b0558f5676427f5a9
Successfully built fasttext
  Attempting uninstall: fasttext
    Found existing installation: fasttext 0.9.2
    Uninstalling fasttext-0.9.2:
      Successfully uninstalled fasttext-0.9.2


In [2]:
%%capture
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

In [3]:
import pandas as pd
import numpy as np
import fasttext
from sklearn.metrics import * 
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords
from string import punctuation
from nltk.tokenize import word_tokenize
import csv
import re
nltk.download('punkt')
#ft_model = fasttext.train_unsupervised('clean_text.txt', minn=3, maxn=4, dim=300)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)

negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)

df = positive.append(negative)
df.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive


In [5]:
df['label']=['__label__'+s for s in df['label']]

df['text']= df['text'].replace('\n',' ', regex=True).replace('\t',' ', regex=True)
df['text'] = df['text'].str.replace(r"http[s]?://(?:[a-zA-Zа-яА-Я]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", ' ', regex=True)
df['text'] = df['text'].str.replace("[^a-zA-Zа-яА-Я]"," ",regex=True)
df['text'] = df['text'].str.lower()

In [6]:
x_train,x_test,y_train,y_test = train_test_split(df.text,df.label,test_size = 0.33)
df1 = pd.DataFrame(data = x_train)
df1['label'] = y_train
df1.to_csv(r'clean_train.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")
df2 = pd.DataFrame(data = x_test)
df2['label'] = y_test
df2.to_csv(r'clean_test.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

In [7]:
ft_model = fasttext.train_supervised('clean_train.txt', minn=2, maxn=4, dim=300)


In [8]:
ft_model.test('clean_test.txt')

(74856, 0.771574756866517, 0.771574756866517)